# LVILC MCMC Interactive Analysis

This notebook provides an interactive interface for running and analyzing MCMC fits to the LoopVacuum Invisible Light Cosmology (LVILC) model.

## Theory

LVILC proposes that:
- The universe exists inside a single giant black hole
- Cosmic expansion is an illusion from horizon dilation
- H₀ ≈ -6.73 ± 0.96 km/s/Mpc (negative in LVILC framework)

## Setup

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from lvilc_model import LVILCModel
from data_loader import CosmologyData
from mcmc_lvilc import LVILC_MCMC
from analysis import MCMCAnalyzer

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Libraries imported successfully!")

## 1. Load Observational Data

In [ ]:
# Load cosmological data
data_loader = CosmologyData()
sn_data = data_loader.load_supernova_data()
bao_data = data_loader.load_bao_data()
cmb_data = data_loader.load_cmb_data()

print(f"Loaded {len(sn_data['z'])} supernova observations")
print(f"Loaded {len(bao_data['z'])} BAO measurements")
print(f"Loaded CMB data")

# Visualize supernova data
plt.figure(figsize=(10, 6))
plt.errorbar(sn_data['z'], sn_data['mu'], yerr=sn_data['mu_err'],
             fmt='o', alpha=0.6, label='Supernova observations')
plt.xlabel('Redshift z', fontsize=12)
plt.ylabel('Distance Modulus μ', fontsize=12)
plt.title('Supernova Data', fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 2. Explore the LVILC Model

In [ ]:
# Create a model with default LVILC parameters
model = LVILCModel(H0=-6.73, M_bh=1e23, t_fall=13.8)

# Calculate predictions at various redshifts
z_range = np.linspace(0.01, 2.0, 100)
d_L = model.luminosity_distance(z_range)
mu = model.distance_modulus(z_range)
H_z = model.hubble_parameter(z_range)

# Plot model predictions
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# Distance modulus
ax1.plot(z_range, mu, 'r-', linewidth=2, label='LVILC Model')
ax1.errorbar(sn_data['z'], sn_data['mu'], yerr=sn_data['mu_err'],
            fmt='o', alpha=0.5, label='Data', color='black')
ax1.set_xlabel('Redshift z', fontsize=12)
ax1.set_ylabel('Distance Modulus μ', fontsize=12)
ax1.set_title('LVILC Model Prediction', fontsize=14)
ax1.legend()
ax1.grid(alpha=0.3)

# Hubble parameter evolution
ax2.plot(z_range, H_z, 'b-', linewidth=2)
ax2.set_xlabel('Redshift z', fontsize=12)
ax2.set_ylabel('H(z) [km/s/Mpc]', fontsize=12)
ax2.set_title('Hubble Parameter Evolution', fontsize=14)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Run MCMC Sampling

Configure and run the MCMC analysis:

In [ ]:
# Configure MCMC parameters
nwalkers = 16
nsteps = 1000
burn_in = 200

# Initial parameters [H0, M_bh, t_fall]
initial_params = np.array([-6.73, 1e23, 13.8])

print(f"Running MCMC with:")
print(f"  Walkers: {nwalkers}")
print(f"  Steps: {nsteps}")
print(f"  Burn-in: {burn_in}")
print(f"  Initial parameters: H0={initial_params[0]}, M_bh={initial_params[1]:.2e}, t_fall={initial_params[2]}")
print("\nThis may take a few minutes...")

In [ ]:
# Initialize and run MCMC
mcmc = LVILC_MCMC(data_loader=data_loader)

sampler = mcmc.run_mcmc(
    initial_params=initial_params,
    nwalkers=nwalkers,
    nsteps=nsteps,
    burn_in=burn_in,
    progress=True
)

print("\nMCMC completed!")

## 4. Analyze Results

In [ ]:
# Extract results
results = mcmc.get_results(sampler)

# Print summary
print("="*60)
print("LVILC MCMC Results")
print("="*60)

for param_name in mcmc.param_names:
    res = results[param_name]
    print(f"\n{param_name}:")
    print(f"  Median: {res['median']:.4e}")
    print(f"  Mean:   {res['mean']:.4e}")
    print(f"  Std:    {res['std']:.4e}")
    print(f"  68% CI: [{res['median'] - res['lower_1sigma']:.4e}, "
          f"{res['median'] + res['upper_1sigma']:.4e}]")

## 5. Visualize Results

In [ ]:
# Create analyzer
analyzer = MCMCAnalyzer(results, data_loader)

# Plot corner plot
fig = analyzer.plot_corner(filename='notebook_corner.png')
plt.show()

print("Corner plot shows parameter constraints and correlations")

In [ ]:
# Plot parameter distributions
fig = analyzer.plot_chains(filename='notebook_chains.png')
plt.show()

print("Chain histograms show parameter distributions")

In [ ]:
# Plot model comparison
fig = analyzer.plot_model_comparison(filename='notebook_comparison.png')
plt.show()

print("Model comparison shows fit to data")

## 6. Goodness of Fit

In [ ]:
# Calculate goodness of fit
gof = analyzer.calculate_goodness_of_fit()

print("\nGoodness of Fit Statistics:")
print(f"  χ² = {gof['chi2']:.2f}")
print(f"  DOF = {gof['dof']}")
print(f"  χ²/DOF = {gof['chi2_reduced']:.2f}")
print(f"  Number of data points = {gof['n_data']}")
print(f"  Number of parameters = {gof['n_params']}")

if gof['chi2_reduced'] < 1.5:
    print("\n✓ Good fit! (χ²/DOF ≈ 1)")
elif gof['chi2_reduced'] < 3.0:
    print("\n⚠ Acceptable fit (χ²/DOF < 3)")
else:
    print("\n✗ Poor fit (χ²/DOF > 3)")

## 7. Explore Parameter Space

Visualize how the model changes with different parameters:

In [ ]:
# Get best-fit parameters
H0_best = results['H0']['median']
M_bh_best = results['M_bh']['median']
t_fall_best = results['t_fall']['median']

# Create models with parameter variations
z_test = np.linspace(0.01, 2.0, 100)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Vary H0
H0_values = [H0_best * 0.8, H0_best, H0_best * 1.2]
for H0 in H0_values:
    model = LVILCModel(H0=H0, M_bh=M_bh_best, t_fall=t_fall_best)
    mu = model.distance_modulus(z_test)
    axes[0].plot(z_test, mu, label=f'H0={H0:.1f}')
axes[0].set_xlabel('Redshift z')
axes[0].set_ylabel('Distance Modulus μ')
axes[0].set_title('Varying H0')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Vary M_bh
M_bh_values = [M_bh_best / 10, M_bh_best, M_bh_best * 10]
for M_bh in M_bh_values:
    model = LVILCModel(H0=H0_best, M_bh=M_bh, t_fall=t_fall_best)
    mu = model.distance_modulus(z_test)
    axes[1].plot(z_test, mu, label=f'M_bh={M_bh:.1e}')
axes[1].set_xlabel('Redshift z')
axes[1].set_ylabel('Distance Modulus μ')
axes[1].set_title('Varying M_bh')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Vary t_fall
t_fall_values = [t_fall_best - 2, t_fall_best, t_fall_best + 2]
for t_fall in t_fall_values:
    model = LVILCModel(H0=H0_best, M_bh=M_bh_best, t_fall=t_fall)
    mu = model.distance_modulus(z_test)
    axes[2].plot(z_test, mu, label=f't_fall={t_fall:.1f}')
axes[2].set_xlabel('Redshift z')
axes[2].set_ylabel('Distance Modulus μ')
axes[2].set_title('Varying t_fall')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook has demonstrated:
1. Loading and visualizing observational data
2. Creating LVILC cosmological models
3. Running MCMC parameter inference
4. Analyzing and visualizing results
5. Exploring parameter space

The LVILC model provides an alternative explanation for cosmic expansion through horizon dilation effects in a universe-scale black hole.